# AKS Cookbook

## 🧪 AI-Driven Security Threat Detection and Response in AKS

![visual](visual.png)

This lab is designed to assist engineers in setting up an AI-driven security threat detection and response system within an Azure Kubernetes Service (AKS) environment. This document includes CLI commands, code snippets, and configuration details to ensure smooth implementation and robust security monitoring.  
This lab based on this [public GitHub repo](https://github.com/gkaleta/AI-Driven-Security-Threat-Detection-and-Response-in-Kubernetes-on-Azure).

▶️ Click on the `Run All` button to execute all the subsequent steps in sequence, or run each step individually by executing the cells one at a time.

### TOC

- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and connected Azure subscription](#1)
- [2️⃣ Create a new Azure Resource Group or reuse an existing one](#2)
- [3️⃣ Create an AKS cluster with an ACR attached](#3)
- [4️⃣ Connect to the AKS cluster](#4)
- [5️⃣ Add Falco Helm Repository](#5)
- [6️⃣ Install Falco](#6)
- [7️⃣ Verify Falco Installation](#7)
- [8️⃣ Develop AI Model (using TensorFlow)](#8)
- [9️⃣ Deploy the AI Model to AKS](#9)
- [🗑️ Clean up resources](#clean)


<a id='0'></a>
### 0️⃣ Initialize notebook variables
Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable).

In [ ]:
import os, time, json, requests, utils

create_resources = True # specify True if you want to create new resources, False to use existing ones

if create_resources:
    # create new resources with the following properties
    deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
    resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
    resource_group_location = "eastus2"
    acr_prefix_name = "acr"
    aks_resource_name = "aks-cluster"
    aks_node_count = 1
else:
    # or use the following existing resources
    resource_group_name = ""
    acr_resource_name = ""
    aks_resource_name = ""


falco_ai_model_deployment = "falco-ai-model.yaml"

utils.print_ok('Notebook initiaized')


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected Azure subscription
The following commands ensure that you have the latest version of the Azure CLI and relevant extensions installed while also verifying that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data.get('user').get('name')
    subscription_id = output.json_data.get('id')
    tenant_id = output.json_data.get('tenantId')
output = utils.run("az provider register --namespace Microsoft.ContainerService --wait", "Microsoft.ContainerService registered in your subscription", "Failed to register Microsoft.ContainerService")
output = utils.run("az provider register --namespace Microsoft.KubernetesConfiguration --wait", "Microsoft.KubernetesConfiguration registered in your subscription", "Failed to register Microsoft.KubernetesConfiguration")
output = utils.run("az extension add --name k8s-extension", "az k8s-extension installed", "Failed to install az k8s-extension")
output = utils.run("az extension update --name k8s-extension", "az k8s-extension updated", "Failed to update az k8s-extension")
output = utils.run("az extension add --name aks-preview", "az aks-preview extension installed", "Failed to install az aks-preview extension")
output = utils.run("az extension update --name aks-preview", "az aks-preview extension updated", "Failed to update az aks-preview extension")

suffix = utils.unique_string(subscription_id + resource_group_name)


<a id='2'></a>
### 2️⃣ Create a new Azure Resource Group or reuse an existing one
All resources deployed in this lab will be created within the designated resource group. 

In [ ]:
if create_resources:
    utils.create_resource_group(create_resources, resource_group_name, resource_group_location)


<a id='3'></a>
### 3️⃣ Create an AKS cluster with an ACR attached
The following step creates an AKS cluster using the [az aks create](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-create) command. 

In [ ]:
if create_resources:
    acr_resource_name = f"{acr_prefix_name}{suffix}"
    output = utils.run(f"az acr create --resource-group {resource_group_name} --name {acr_resource_name} --sku basic", 
            f"ACR '{acr_resource_name}' created",
            f"Failed to create ACR '{acr_resource_name}")
    if output.success:
        output = utils.run(f"az aks create --resource-group {resource_group_name} --name {aks_resource_name} --node-count {aks_node_count} --enable-addons monitoring --network-policy azure --network-plugin azure --generate-ssh-keys --attach-acr {acr_resource_name}",
                f"AKS cluster '{aks_resource_name}' created",
                f"Failed to create AKS cluster '{aks_resource_name}'")

<a id='4'></a>
### 4️⃣ Connect to the AKS cluster
Configure kubectl to connect to your Kubernetes cluster using the [az aks get-credentials](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-get-credentials) command. This command downloads credentials and configures the Kubernetes CLI to use them.

In [ ]:
output = utils.run("az aks install-cli", "kubectl installed", "Failed to install kubectl")
output = utils.run(f"az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing",
             f"Credentials for AKS cluster '{aks_resource_name}' configured",
             f"Failed to configure credentials for AKS cluster '{aks_resource_name}'")


<a id='5'></a>
### 5️⃣ Add Falco Helm Repository

Add the Falco repository to Helm and update your repository list.



In [ ]:
! helm repo add falcosecurity https://falcosecurity.github.io/charts
! helm repo update

<a id='6'></a>
### 6️⃣ Install Falco

Install Falco in your AKS cluster.

In [ ]:
! helm install falco falcosecurity/falco --namespace falco --create-namespace

<a id='7'></a>
### 7️⃣ Verify Falco Installation

Check that Falco is running correctly and view the logs with the following command to ensure it's monitoring your system:  
`kubectl logs <falco-pod-name> -n falco`


In [ ]:
! kubectl get pods -n falco

# If you need custom detection rules, you can edit the Falco configuration.
# ! kubectl edit cm falco-rules -n falco

<a id='8'></a>
### 8️⃣ Develop AI Model (using TensorFlow)

Create and train an AI model to detect anomalies.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Example: Load and preprocess data
data = np.random.rand(1000, 20)  # Replace with actual log data

# Define a simple model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(20,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(data, np.ones((1000, 1)), epochs=10)

# Save the model
model.save('falco_anomaly_model.h5')

<a id='9'></a>
### 9️⃣ Deploy the AI Model to AKS

In [ ]:
utils.run(f"az acr build --image falco-ai-model:latest --resource-group {resource_group_name} --registry {acr_resource_name} --file Dockerfile . --no-logs", "Image was successfully built", "Failed to build the image")

utils.file_string_interpolation(falco_ai_model_deployment, ".", ".temp", acr_resource_name=acr_resource_name)

utils.run(f"kubectl apply -f .temp/{falco_ai_model_deployment}", "Falco AI model deployment created", "Failed to create Falco AI model deployment")

# Apply the network policy to enforce the rules in your AKS cluster.
utils.run(f"kubectl apply -f network-policy.yaml", "Network policy deployment created", "Failed to create Network policy deployment")



<a id='clean'></a>
### 🗑️ Clean up resources
When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered. Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.